## Домашнее задание 2

1) Установить PostgreSQL;
2) Создать БД home_credit, исследовать данные, определить типы данных и загрузить все таблицы. SQL запросы указать либо в ноутбуке, либо отдельными файлами;
3) Сделать упражнения по запросам ниже;
4) Сгенерировать 5 своих признаков для будущего обучения модели;
5) Реализовать их насчет используя ТОЛЬКО SQL-запросы.

Результат:
1) Настроенная СУБД с загруженными данными из конкурса Home_Credit;
2) Сделанные упражнения с выводом данных как в примерах выше;
3) Описание в ноутбуке 5 сгенерированных признаков;
4) Реализованные в ноутбуке SQL-запросы к базе данных с выводом результата по вышеуказанным фичам;
5) Файл конфигурации в папке config, где прописаны параметры коннекта к БД;
6) Папка utils в репозиторий, созданный там скрипт с кодом позволяющим обращаться к БД;

Бонусные задания:
1) Сделать класс для операций с БД вместо функций;
2) Проанализировать структуру, назначить в таблицах ключи и индексы запросами к БД.

In [1]:
import sys
sys.path.insert(1, '/Users/vi/DataspellProjects/shift/src/app/utils')
import db_connector
sys.path.insert(1, '/Users/vi/DataspellProjects/shift/src/config')
import db_params

In [2]:
connector = db_connector.database_connector(db_params.DB_ARGS)

## Анализ типов переменных

In [3]:
import pandas as pd

In [4]:
file_path = db_params.DATA_FULL_PATH + 'application_test.csv'
application_test = pd.read_csv(file_path)

In [5]:
file_path = db_params.DATA_FULL_PATH + 'application_train.csv'
application_train = pd.read_csv(file_path)

In [6]:
application = pd.concat([application_test, application_train], ignore_index=True)

In [7]:
application.dtypes

SK_ID_CURR                      int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
                               ...   
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
TARGET                        float64
Length: 122, dtype: object

In [8]:
type_object= application.select_dtypes(include='object').columns
type_varchar = {}
for column in type_object:
    max_length = application[column].dropna().astype(str).map(len).max()
    type_varchar[column] = max_length + 5
print(type_varchar)

{'NAME_CONTRACT_TYPE': 20, 'CODE_GENDER': 8, 'FLAG_OWN_CAR': 6, 'FLAG_OWN_REALTY': 6, 'NAME_TYPE_SUITE': 20, 'NAME_INCOME_TYPE': 25, 'NAME_EDUCATION_TYPE': 34, 'NAME_FAMILY_STATUS': 25, 'NAME_HOUSING_TYPE': 24, 'OCCUPATION_TYPE': 26, 'WEEKDAY_APPR_PROCESS_START': 14, 'ORGANIZATION_TYPE': 27, 'FONDKAPREMONT_MODE': 26, 'HOUSETYPE_MODE': 21, 'WALLSMATERIAL_MODE': 17, 'EMERGENCYSTATE_MODE': 8}


In [9]:
type_integer = {}
for column in application.columns:
    if application[column].dtype == 'int64':
        maximum = application[column].max()
        minimum = application[column].min()
        # Определяем, подходит ли тип smallint, используя границы значений.
        if maximum < 2**15 and minimum > -(2**15):
            type_integer[column] = 'smallint'
        else:
            type_integer[column] = 'integer'
print(type_integer)

{'SK_ID_CURR': 'integer', 'CNT_CHILDREN': 'smallint', 'DAYS_BIRTH': 'smallint', 'DAYS_EMPLOYED': 'integer', 'DAYS_ID_PUBLISH': 'smallint', 'FLAG_MOBIL': 'smallint', 'FLAG_EMP_PHONE': 'smallint', 'FLAG_WORK_PHONE': 'smallint', 'FLAG_CONT_MOBILE': 'smallint', 'FLAG_PHONE': 'smallint', 'FLAG_EMAIL': 'smallint', 'REGION_RATING_CLIENT': 'smallint', 'REGION_RATING_CLIENT_W_CITY': 'smallint', 'HOUR_APPR_PROCESS_START': 'smallint', 'REG_REGION_NOT_LIVE_REGION': 'smallint', 'REG_REGION_NOT_WORK_REGION': 'smallint', 'LIVE_REGION_NOT_WORK_REGION': 'smallint', 'REG_CITY_NOT_LIVE_CITY': 'smallint', 'REG_CITY_NOT_WORK_CITY': 'smallint', 'LIVE_CITY_NOT_WORK_CITY': 'smallint', 'FLAG_DOCUMENT_2': 'smallint', 'FLAG_DOCUMENT_3': 'smallint', 'FLAG_DOCUMENT_4': 'smallint', 'FLAG_DOCUMENT_5': 'smallint', 'FLAG_DOCUMENT_6': 'smallint', 'FLAG_DOCUMENT_7': 'smallint', 'FLAG_DOCUMENT_8': 'smallint', 'FLAG_DOCUMENT_9': 'smallint', 'FLAG_DOCUMENT_10': 'smallint', 'FLAG_DOCUMENT_11': 'smallint', 'FLAG_DOCUMENT_12'

## Создание БД

In [10]:
sql_schema_query = """
      SET search_path TO credit_scoring;
      DROP TABLE IF EXISTS bureau; 
      CREATE TABLE bureau(
      SK_ID_CURR integer,
      SK_ID_BUREAU integer,
      CREDIT_ACTIVE varchar(13),
      CREDIT_CURRENCY varchar(15),
      DAYS_CREDIT smallint,
      CREDIT_DAY_OVERDUE smallint,
      DAYS_CREDIT_ENDDATE real,
      DAYS_ENDDATE_FACT real,
      AMT_CREDIT_MAX_OVERDUE real,
      CNT_CREDIT_PROLONG smallint,
      AMT_CREDIT_SUM real,
      AMT_CREDIT_SUM_DEBT real,
      AMT_CREDIT_SUM_LIMIT real,
      AMT_CREDIT_SUM_OVERDUE real,
      CREDIT_TYPE varchar(49),
      DAYS_CREDIT_UPDATE integer,
      AMT_ANNUITY  real)
"""

In [11]:
%%time
connector.send_sql_query(sql_schema_query)

CPU times: user 10.8 ms, sys: 9.62 ms, total: 20.4 ms
Wall time: 197 ms


In [12]:
sql_schema_query = """
    SET search_path TO credit_scoring;
    DROP TABLE IF EXISTS application;
    CREATE TABLE application(
        SK_ID_CURR integer,
        TARGET real,
        NAME_CONTRACT_TYPE varchar(20),
        CODE_GENDER varchar(8),
        FLAG_OWN_CAR varchar(6),
        FLAG_OWN_REALTY varchar(6),
        CNT_CHILDREN smallint,
        AMT_INCOME_TOTAL real,
        AMT_CREDIT real,
        AMT_ANNUITY real,
        AMT_GOODS_PRICE real,
        NAME_TYPE_SUITE varchar(20),
        NAME_INCOME_TYPE varchar(25),
        NAME_EDUCATION_TYPE varchar(34),
        NAME_FAMILY_STATUS varchar(25),
        NAME_HOUSING_TYPE varchar(24),
        REGION_POPULATION_RELATIVE real,
        DAYS_BIRTH smallint,
        DAYS_EMPLOYED integer,
        DAYS_REGISTRATION real,
        DAYS_ID_PUBLISH smallint,
        OWN_CAR_AGE real,
        FLAG_MOBIL smallint,
        FLAG_EMP_PHONE smallint,
        FLAG_WORK_PHONE smallint,
        FLAG_CONT_MOBILE smallint,
        FLAG_PHONE smallint,
        FLAG_EMAIL smallint,
        OCCUPATION_TYPE varchar(26),
        CNT_FAM_MEMBERS real,
        REGION_RATING_CLIENT smallint,
        REGION_RATING_CLIENT_W_CITY smallint,
        WEEKDAY_APPR_PROCESS_START varchar(14),
        HOUR_APPR_PROCESS_START smallint,
        REG_REGION_NOT_LIVE_REGION smallint,
        REG_REGION_NOT_WORK_REGION smallint,
        LIVE_REGION_NOT_WORK_REGION smallint,
        REG_CITY_NOT_LIVE_CITY smallint,
        REG_CITY_NOT_WORK_CITY smallint,
        LIVE_CITY_NOT_WORK_CITY smallint,
        ORGANIZATION_TYPE varchar(27),
        EXT_SOURCE_1 real,
        EXT_SOURCE_2 real,
        EXT_SOURCE_3 real,
        APARTMENTS_AVG real,
        BASEMENTAREA_AVG real,
        YEARS_BEGINEXPLUATATION_AVG real,
        YEARS_BUILD_AVG real,
        COMMONAREA_AVG real,
        ELEVATORS_AVG real,
        ENTRANCES_AVG real,
        FLOORSMAX_AVG real,
        FLOORSMIN_AVG real,
        LANDAREA_AVG real,
        LIVINGAPARTMENTS_AVG real,
        LIVINGAREA_AVG real,
        NONLIVINGAPARTMENTS_AVG real,
        NONLIVINGAREA_AVG real,
        APARTMENTS_MODE real,
        BASEMENTAREA_MODE real,
        YEARS_BEGINEXPLUATATION_MODE real,
        YEARS_BUILD_MODE real,
        COMMONAREA_MODE real,
        ELEVATORS_MODE real,
        ENTRANCES_MODE real,
        FLOORSMAX_MODE real,
        FLOORSMIN_MODE real,
        LANDAREA_MODE real,
        LIVINGAPARTMENTS_MODE real,
        LIVINGAREA_MODE real,
        NONLIVINGAPARTMENTS_MODE real,
        NONLIVINGAREA_MODE real,
        APARTMENTS_MEDI real,
        BASEMENTAREA_MEDI real,
        YEARS_BEGINEXPLUATATION_MEDI real,
        YEARS_BUILD_MEDI real,
        COMMONAREA_MEDI real,
        ELEVATORS_MEDI real,
        ENTRANCES_MEDI real,
        FLOORSMAX_MEDI real,
        FLOORSMIN_MEDI real,
        LANDAREA_MEDI real,
        LIVINGAPARTMENTS_MEDI real,
        LIVINGAREA_MEDI real,
        NONLIVINGAPARTMENTS_MEDI real,
        NONLIVINGAREA_MEDI real,
        FONDKAPREMONT_MODE varchar(26),
        HOUSETYPE_MODE varchar(21),
        TOTALAREA_MODE real,
        WALLSMATERIAL_MODE varchar(17),
        EMERGENCYSTATE_MODE varchar(8),
        OBS_30_CNT_SOCIAL_CIRCLE real,
        DEF_30_CNT_SOCIAL_CIRCLE real,
        OBS_60_CNT_SOCIAL_CIRCLE real,
        DEF_60_CNT_SOCIAL_CIRCLE real,
        DAYS_LAST_PHONE_CHANGE real,
        FLAG_DOCUMENT_2 smallint,
        FLAG_DOCUMENT_3 smallint,
        FLAG_DOCUMENT_4 smallint,
        FLAG_DOCUMENT_5 smallint,
        FLAG_DOCUMENT_6 smallint,
        FLAG_DOCUMENT_7 smallint,
        FLAG_DOCUMENT_8 smallint,
        FLAG_DOCUMENT_9 smallint,
        FLAG_DOCUMENT_10 smallint,
        FLAG_DOCUMENT_11 smallint,
        FLAG_DOCUMENT_12 smallint,
        FLAG_DOCUMENT_13 smallint,
        FLAG_DOCUMENT_14 smallint,
        FLAG_DOCUMENT_15 smallint,
        FLAG_DOCUMENT_16 smallint,
        FLAG_DOCUMENT_17 smallint,
        FLAG_DOCUMENT_18 smallint,
        FLAG_DOCUMENT_19 smallint,
        FLAG_DOCUMENT_20 smallint,
        FLAG_DOCUMENT_21 smallint,
        AMT_REQ_CREDIT_BUREAU_HOUR real,
        AMT_REQ_CREDIT_BUREAU_DAY real,
        AMT_REQ_CREDIT_BUREAU_WEEK real,
        AMT_REQ_CREDIT_BUREAU_MON real,
        AMT_REQ_CREDIT_BUREAU_QRT real,
        AMT_REQ_CREDIT_BUREAU_YEAR real)
"""

In [13]:
%%time
connector.send_sql_query(sql_schema_query)

CPU times: user 5.53 ms, sys: 4.17 ms, total: 9.7 ms
Wall time: 54.5 ms


In [14]:
sql_schema_query = """
    SET search_path TO credit_scoring;
    DROP TABLE IF EXISTS bureau_balance;
    CREATE TABLE bureau_balance(
        SK_ID_BUREAU integer,
        MONTHS_BALANCE smallint,
        STATUS varchar(6))
"""

In [15]:
%%time
connector.send_sql_query(sql_schema_query)

CPU times: user 6.28 ms, sys: 3.85 ms, total: 10.1 ms
Wall time: 70.4 ms


In [16]:
sql_schema_query = """
    SET search_path TO credit_scoring;
    DROP TABLE IF EXISTS previous_application;
    CREATE TABLE previous_application(
        SK_ID_PREV integer,
        SK_ID_CURR integer,
        NAME_CONTRACT_TYPE varchar(20),
        AMT_ANNUITY real,
        AMT_APPLICATION real,
        AMT_CREDIT real,
        AMT_DOWN_PAYMENT real,
        AMT_GOODS_PRICE real,
        WEEKDAY_APPR_PROCESS_START varchar(14),
        HOUR_APPR_PROCESS_START smallint,
        FLAG_LAST_APPL_PER_CONTRACT varchar(6),
        NFLAG_LAST_APPL_IN_DAY smallint,
        RATE_DOWN_PAYMENT real,
        RATE_INTEREST_PRIMARY real,
        RATE_INTEREST_PRIVILEGED real,
        NAME_CASH_LOAN_PURPOSE varchar(37),
        NAME_CONTRACT_STATUS varchar(17),
        DAYS_DECISION smallint,
        NAME_PAYMENT_TYPE varchar(46),
        CODE_REJECT_REASON varchar(11),
        NAME_TYPE_SUITE varchar(20),
        NAME_CLIENT_TYPE varchar(14),
        NAME_GOODS_CATEGORY varchar(29),
        NAME_PORTFOLIO varchar(10),
        NAME_PRODUCT_TYPE varchar(12),
        CHANNEL_TYPE varchar(31),
        SELLERPLACE_AREA integer,
        NAME_SELLER_INDUSTRY varchar(25),
        CNT_PAYMENT real,
        NAME_YIELD_GROUP varchar(15),
        PRODUCT_COMBINATION varchar(35),
        DAYS_FIRST_DRAWING real,
        DAYS_FIRST_DUE real,
        DAYS_LAST_DUE_1ST_VERSION real,
        DAYS_LAST_DUE real,
        DAYS_TERMINATION real,
        NFLAG_INSURED_ON_APPROVAL real)   
"""

In [17]:
%%time
connector.send_sql_query(sql_schema_query)

CPU times: user 5.29 ms, sys: 3.81 ms, total: 9.1 ms
Wall time: 54.6 ms


In [18]:
sql_schema_query = """
    SET search_path TO credit_scoring;
    DROP TABLE IF EXISTS POS_CASH_balance;
    CREATE TABLE POS_CASH_balance(
       SK_ID_PREV integer,    
       SK_ID_CURR integer,
       MONTHS_BALANCE smallint,
       CNT_INSTALMENT real,
       CNT_INSTALMENT_FUTURE real,
       NAME_CONTRACT_STATUS varchar(26),
       SK_DPD smallint,
       SK_DPD_DEF smallint)
"""

In [19]:
%%time
connector.send_sql_query(sql_schema_query)

CPU times: user 5.89 ms, sys: 4.17 ms, total: 10.1 ms
Wall time: 72.7 ms


In [20]:
sql_schema_query = """
    SET search_path TO credit_scoring;
    DROP TABLE IF EXISTS credit_card_balance;
    CREATE TABLE credit_card_balance(
       SK_ID_PREV integer,
       SK_ID_CURR integer,
       MONTHS_BALANCE smallint,
       AMT_BALANCE real,
       AMT_CREDIT_LIMIT_ACTUAL integer,
       AMT_DRAWINGS_ATM_CURRENT real,
       AMT_DRAWINGS_CURRENT real,
       AMT_DRAWINGS_OTHER_CURRENT real,
       AMT_DRAWINGS_POS_CURRENT real,
       AMT_INST_MIN_REGULARITY real,
       AMT_PAYMENT_CURRENT real,
       AMT_PAYMENT_TOTAL_CURRENT real,
       AMT_RECEIVABLE_PRINCIPAL real,
       AMT_RECIVABLE real,
       AMT_TOTAL_RECEIVABLE real,
       CNT_DRAWINGS_ATM_CURRENT real,
       CNT_DRAWINGS_CURRENT smallint,
       CNT_DRAWINGS_OTHER_CURRENT real,
       CNT_DRAWINGS_POS_CURRENT real,
       CNT_INSTALMENT_MATURE_CUM real,
       NAME_CONTRACT_STATUS varchar(18),
       SK_DPD smallint,
       SK_DPD_DEF smallint)
"""

In [21]:
%%time
connector.send_sql_query(sql_schema_query)

CPU times: user 6.46 ms, sys: 3.81 ms, total: 10.3 ms
Wall time: 54 ms


In [22]:
sql_schema_query = """
    SET search_path TO credit_scoring;
    DROP TABLE IF EXISTS installments_payments;
    CREATE TABLE installments_payments(
       SK_ID_PREV integer,
       SK_ID_CURR integer,
       NUM_INSTALMENT_VERSION real,
       NUM_INSTALMENT_NUMBER smallint,
       DAYS_INSTALMENT real,
       DAYS_ENTRY_PAYMENT real,
       AMT_INSTALMENT real,
       AMT_PAYMENT real)

"""

In [23]:
%%time
connector.send_sql_query(sql_schema_query)

CPU times: user 5.75 ms, sys: 4.09 ms, total: 9.84 ms
Wall time: 32.9 ms


## Наполнение БД

In [24]:
sql_data_query = f"""
SET search_path TO credit_scoring;
COPY bureau(
    SK_ID_CURR,
    SK_ID_BUREAU,
    CREDIT_ACTIVE,
    CREDIT_CURRENCY,
    DAYS_CREDIT,
    CREDIT_DAY_OVERDUE,
    DAYS_CREDIT_ENDDATE,
    DAYS_ENDDATE_FACT,
    AMT_CREDIT_MAX_OVERDUE,
    CNT_CREDIT_PROLONG,
    AMT_CREDIT_SUM,
    AMT_CREDIT_SUM_DEBT,
    AMT_CREDIT_SUM_LIMIT,
    AMT_CREDIT_SUM_OVERDUE,
    CREDIT_TYPE,
    DAYS_CREDIT_UPDATE,
    AMT_ANNUITY
)
FROM '{db_params.DATA_FULL_PATH}bureau.csv' DELIMITER ',' CSV HEADER;
"""

In [25]:
%%time
connector.send_sql_query(sql_data_query)

CPU times: user 7.49 ms, sys: 4.77 ms, total: 12.3 ms
Wall time: 27.2 s


In [26]:
sql_data_query = f"""
SET search_path TO credit_scoring;
COPY application(
 SK_ID_CURR,
 NAME_CONTRACT_TYPE,
 CODE_GENDER,
 FLAG_OWN_CAR,
 FLAG_OWN_REALTY,
 CNT_CHILDREN,
 AMT_INCOME_TOTAL,
 AMT_CREDIT,
 AMT_ANNUITY,
 AMT_GOODS_PRICE,
 NAME_TYPE_SUITE,
 NAME_INCOME_TYPE,
 NAME_EDUCATION_TYPE,
 NAME_FAMILY_STATUS,
 NAME_HOUSING_TYPE,
 REGION_POPULATION_RELATIVE,
 DAYS_BIRTH,
 DAYS_EMPLOYED,
 DAYS_REGISTRATION,
 DAYS_ID_PUBLISH,
 OWN_CAR_AGE,
 FLAG_MOBIL,
 FLAG_EMP_PHONE,
 FLAG_WORK_PHONE,
 FLAG_CONT_MOBILE,
 FLAG_PHONE,
 FLAG_EMAIL,
 OCCUPATION_TYPE,
 CNT_FAM_MEMBERS,
 REGION_RATING_CLIENT,
 REGION_RATING_CLIENT_W_CITY,
 WEEKDAY_APPR_PROCESS_START,
 HOUR_APPR_PROCESS_START,
 REG_REGION_NOT_LIVE_REGION,
 REG_REGION_NOT_WORK_REGION,
 LIVE_REGION_NOT_WORK_REGION,
 REG_CITY_NOT_LIVE_CITY,
 REG_CITY_NOT_WORK_CITY,
 LIVE_CITY_NOT_WORK_CITY,
 ORGANIZATION_TYPE,
 EXT_SOURCE_1,
 EXT_SOURCE_2,
 EXT_SOURCE_3,
 APARTMENTS_AVG,
 BASEMENTAREA_AVG,
 YEARS_BEGINEXPLUATATION_AVG,
 YEARS_BUILD_AVG,
 COMMONAREA_AVG,
 ELEVATORS_AVG,
 ENTRANCES_AVG,
 FLOORSMAX_AVG,
 FLOORSMIN_AVG,
 LANDAREA_AVG,
 LIVINGAPARTMENTS_AVG,
 LIVINGAREA_AVG,
 NONLIVINGAPARTMENTS_AVG,
 NONLIVINGAREA_AVG,
 APARTMENTS_MODE,
 BASEMENTAREA_MODE,
 YEARS_BEGINEXPLUATATION_MODE,
 YEARS_BUILD_MODE,
 COMMONAREA_MODE,
 ELEVATORS_MODE,
 ENTRANCES_MODE,
 FLOORSMAX_MODE,
 FLOORSMIN_MODE,
 LANDAREA_MODE,
 LIVINGAPARTMENTS_MODE,
 LIVINGAREA_MODE,
 NONLIVINGAPARTMENTS_MODE,
 NONLIVINGAREA_MODE,
 APARTMENTS_MEDI,
 BASEMENTAREA_MEDI,
 YEARS_BEGINEXPLUATATION_MEDI,
 YEARS_BUILD_MEDI,
 COMMONAREA_MEDI,
 ELEVATORS_MEDI,
 ENTRANCES_MEDI,
 FLOORSMAX_MEDI,
 FLOORSMIN_MEDI,
 LANDAREA_MEDI,
 LIVINGAPARTMENTS_MEDI,
 LIVINGAREA_MEDI,
 NONLIVINGAPARTMENTS_MEDI,
 NONLIVINGAREA_MEDI,
 FONDKAPREMONT_MODE,
 HOUSETYPE_MODE,
 TOTALAREA_MODE,
 WALLSMATERIAL_MODE,
 EMERGENCYSTATE_MODE,
 OBS_30_CNT_SOCIAL_CIRCLE,
 DEF_30_CNT_SOCIAL_CIRCLE,
 OBS_60_CNT_SOCIAL_CIRCLE,
 DEF_60_CNT_SOCIAL_CIRCLE,
 DAYS_LAST_PHONE_CHANGE,
 FLAG_DOCUMENT_2,
 FLAG_DOCUMENT_3,
 FLAG_DOCUMENT_4,
 FLAG_DOCUMENT_5,
 FLAG_DOCUMENT_6,
 FLAG_DOCUMENT_7,
 FLAG_DOCUMENT_8,
 FLAG_DOCUMENT_9,
 FLAG_DOCUMENT_10,
 FLAG_DOCUMENT_11,
 FLAG_DOCUMENT_12,
 FLAG_DOCUMENT_13,
 FLAG_DOCUMENT_14,
 FLAG_DOCUMENT_15,
 FLAG_DOCUMENT_16,
 FLAG_DOCUMENT_17,
 FLAG_DOCUMENT_18,
 FLAG_DOCUMENT_19,
 FLAG_DOCUMENT_20,
 FLAG_DOCUMENT_21,
 AMT_REQ_CREDIT_BUREAU_HOUR,
 AMT_REQ_CREDIT_BUREAU_DAY,
 AMT_REQ_CREDIT_BUREAU_WEEK,
 AMT_REQ_CREDIT_BUREAU_MON,
 AMT_REQ_CREDIT_BUREAU_QRT,
 AMT_REQ_CREDIT_BUREAU_YEAR
)
FROM '{db_params.DATA_FULL_PATH}application_test.csv' DELIMITER ',' CSV HEADER;
"""

In [27]:
%%time
connector.send_sql_query(sql_data_query)

CPU times: user 6.24 ms, sys: 4.95 ms, total: 11.2 ms
Wall time: 2.5 s


In [28]:
sql_data_query = f"""
SET search_path TO credit_scoring;
COPY application(
 SK_ID_CURR,
 TARGET,
 NAME_CONTRACT_TYPE,
 CODE_GENDER,
 FLAG_OWN_CAR,
 FLAG_OWN_REALTY,
 CNT_CHILDREN,
 AMT_INCOME_TOTAL,
 AMT_CREDIT,
 AMT_ANNUITY,
 AMT_GOODS_PRICE,
 NAME_TYPE_SUITE,
 NAME_INCOME_TYPE,
 NAME_EDUCATION_TYPE,
 NAME_FAMILY_STATUS,
 NAME_HOUSING_TYPE,
 REGION_POPULATION_RELATIVE,
 DAYS_BIRTH,
 DAYS_EMPLOYED,
 DAYS_REGISTRATION,
 DAYS_ID_PUBLISH,
 OWN_CAR_AGE,
 FLAG_MOBIL,
 FLAG_EMP_PHONE,
 FLAG_WORK_PHONE,
 FLAG_CONT_MOBILE,
 FLAG_PHONE,
 FLAG_EMAIL,
 OCCUPATION_TYPE,
 CNT_FAM_MEMBERS,
 REGION_RATING_CLIENT,
 REGION_RATING_CLIENT_W_CITY,
 WEEKDAY_APPR_PROCESS_START,
 HOUR_APPR_PROCESS_START,
 REG_REGION_NOT_LIVE_REGION,
 REG_REGION_NOT_WORK_REGION,
 LIVE_REGION_NOT_WORK_REGION,
 REG_CITY_NOT_LIVE_CITY,
 REG_CITY_NOT_WORK_CITY,
 LIVE_CITY_NOT_WORK_CITY,
 ORGANIZATION_TYPE,
 EXT_SOURCE_1,
 EXT_SOURCE_2,
 EXT_SOURCE_3,
 APARTMENTS_AVG,
 BASEMENTAREA_AVG,
 YEARS_BEGINEXPLUATATION_AVG,
 YEARS_BUILD_AVG,
 COMMONAREA_AVG,
 ELEVATORS_AVG,
 ENTRANCES_AVG,
 FLOORSMAX_AVG,
 FLOORSMIN_AVG,
 LANDAREA_AVG,
 LIVINGAPARTMENTS_AVG,
 LIVINGAREA_AVG,
 NONLIVINGAPARTMENTS_AVG,
 NONLIVINGAREA_AVG,
 APARTMENTS_MODE,
 BASEMENTAREA_MODE,
 YEARS_BEGINEXPLUATATION_MODE,
 YEARS_BUILD_MODE,
 COMMONAREA_MODE,
 ELEVATORS_MODE,
 ENTRANCES_MODE,
 FLOORSMAX_MODE,
 FLOORSMIN_MODE,
 LANDAREA_MODE,
 LIVINGAPARTMENTS_MODE,
 LIVINGAREA_MODE,
 NONLIVINGAPARTMENTS_MODE,
 NONLIVINGAREA_MODE,
 APARTMENTS_MEDI,
 BASEMENTAREA_MEDI,
 YEARS_BEGINEXPLUATATION_MEDI,
 YEARS_BUILD_MEDI,
 COMMONAREA_MEDI,
 ELEVATORS_MEDI,
 ENTRANCES_MEDI,
 FLOORSMAX_MEDI,
 FLOORSMIN_MEDI,
 LANDAREA_MEDI,
 LIVINGAPARTMENTS_MEDI,
 LIVINGAREA_MEDI,
 NONLIVINGAPARTMENTS_MEDI,
 NONLIVINGAREA_MEDI,
 FONDKAPREMONT_MODE,
 HOUSETYPE_MODE,
 TOTALAREA_MODE,
 WALLSMATERIAL_MODE,
 EMERGENCYSTATE_MODE,
 OBS_30_CNT_SOCIAL_CIRCLE,
 DEF_30_CNT_SOCIAL_CIRCLE,
 OBS_60_CNT_SOCIAL_CIRCLE,
 DEF_60_CNT_SOCIAL_CIRCLE,
 DAYS_LAST_PHONE_CHANGE,
 FLAG_DOCUMENT_2,
 FLAG_DOCUMENT_3,
 FLAG_DOCUMENT_4,
 FLAG_DOCUMENT_5,
 FLAG_DOCUMENT_6,
 FLAG_DOCUMENT_7,
 FLAG_DOCUMENT_8,
 FLAG_DOCUMENT_9,
 FLAG_DOCUMENT_10,
 FLAG_DOCUMENT_11,
 FLAG_DOCUMENT_12,
 FLAG_DOCUMENT_13,
 FLAG_DOCUMENT_14,
 FLAG_DOCUMENT_15,
 FLAG_DOCUMENT_16,
 FLAG_DOCUMENT_17,
 FLAG_DOCUMENT_18,
 FLAG_DOCUMENT_19,
 FLAG_DOCUMENT_20,
 FLAG_DOCUMENT_21,
 AMT_REQ_CREDIT_BUREAU_HOUR,
 AMT_REQ_CREDIT_BUREAU_DAY,
 AMT_REQ_CREDIT_BUREAU_WEEK,
 AMT_REQ_CREDIT_BUREAU_MON,
 AMT_REQ_CREDIT_BUREAU_QRT,
 AMT_REQ_CREDIT_BUREAU_YEAR
)
FROM '{db_params.DATA_FULL_PATH}application_train.csv' DELIMITER ',' CSV HEADER;
"""

In [29]:
%%time
connector.send_sql_query(sql_data_query)

CPU times: user 8.06 ms, sys: 4.95 ms, total: 13 ms
Wall time: 17.8 s


In [30]:
sql_data_query = f"""
SET search_path TO credit_scoring;
COPY installments_payments(
  SK_ID_PREV,
  SK_ID_CURR,
  NUM_INSTALMENT_VERSION,
  NUM_INSTALMENT_NUMBER,
  DAYS_INSTALMENT,
  DAYS_ENTRY_PAYMENT,
  AMT_INSTALMENT,
  AMT_PAYMENT
)
FROM '{db_params.DATA_FULL_PATH}installments_payments.csv' DELIMITER ',' CSV HEADER;
"""

In [31]:
%%time
connector.send_sql_query(sql_data_query)

CPU times: user 15.2 ms, sys: 10.4 ms, total: 25.6 ms
Wall time: 1min 50s


In [32]:
sql_data_query = f"""
SET search_path TO credit_scoring;
COPY credit_card_balance(
  SK_ID_PREV,
  SK_ID_CURR,
  MONTHS_BALANCE,
  AMT_BALANCE,
  AMT_CREDIT_LIMIT_ACTUAL,
  AMT_DRAWINGS_ATM_CURRENT,
  AMT_DRAWINGS_CURRENT,
  AMT_DRAWINGS_OTHER_CURRENT,
  AMT_DRAWINGS_POS_CURRENT,
  AMT_INST_MIN_REGULARITY,
  AMT_PAYMENT_CURRENT,
  AMT_PAYMENT_TOTAL_CURRENT,
  AMT_RECEIVABLE_PRINCIPAL,
  AMT_RECIVABLE,
  AMT_TOTAL_RECEIVABLE,
  CNT_DRAWINGS_ATM_CURRENT,
  CNT_DRAWINGS_CURRENT,
  CNT_DRAWINGS_OTHER_CURRENT,
  CNT_DRAWINGS_POS_CURRENT,
  CNT_INSTALMENT_MATURE_CUM,
  NAME_CONTRACT_STATUS,
  SK_DPD,
  SK_DPD_DEF
)
FROM '{db_params.DATA_FULL_PATH}credit_card_balance.csv' DELIMITER ',' CSV HEADER;
"""

In [33]:
%%time
connector.send_sql_query(sql_data_query)

CPU times: user 9.83 ms, sys: 10.8 ms, total: 20.6 ms
Wall time: 38.7 s


In [34]:
sql_data_query = f"""
SET search_path TO credit_scoring;
COPY POS_CASH_balance(
  SK_ID_PREV,
  SK_ID_CURR,
  MONTHS_BALANCE,
  CNT_INSTALMENT,
  CNT_INSTALMENT_FUTURE,
  NAME_CONTRACT_STATUS,
  SK_DPD,
  SK_DPD_DEF
)
FROM '{db_params.DATA_FULL_PATH}POS_CASH_balance.csv' DELIMITER ',' CSV HEADER;
"""

In [35]:
%%time
connector.send_sql_query(sql_data_query)

CPU times: user 4.81 ms, sys: 3.82 ms, total: 8.63 ms
Wall time: 33.7 s


In [36]:
sql_data_query = f"""
SET search_path TO credit_scoring;
COPY previous_application(
  SK_ID_PREV,
  SK_ID_CURR,
  NAME_CONTRACT_TYPE,
  AMT_ANNUITY,
  AMT_APPLICATION,
  AMT_CREDIT,
  AMT_DOWN_PAYMENT,
  AMT_GOODS_PRICE,
  WEEKDAY_APPR_PROCESS_START,
  HOUR_APPR_PROCESS_START,
  FLAG_LAST_APPL_PER_CONTRACT,
  NFLAG_LAST_APPL_IN_DAY,
  RATE_DOWN_PAYMENT,
  RATE_INTEREST_PRIMARY,
  RATE_INTEREST_PRIVILEGED,
  NAME_CASH_LOAN_PURPOSE,
  NAME_CONTRACT_STATUS,
  DAYS_DECISION,
  NAME_PAYMENT_TYPE,
  CODE_REJECT_REASON,
  NAME_TYPE_SUITE,
  NAME_CLIENT_TYPE,
  NAME_GOODS_CATEGORY,
  NAME_PORTFOLIO,
  NAME_PRODUCT_TYPE,
  CHANNEL_TYPE,
  SELLERPLACE_AREA,
  NAME_SELLER_INDUSTRY,
  CNT_PAYMENT,
  NAME_YIELD_GROUP,
  PRODUCT_COMBINATION,
  DAYS_FIRST_DRAWING,
  DAYS_FIRST_DUE,
  DAYS_LAST_DUE_1ST_VERSION,
  DAYS_LAST_DUE,
  DAYS_TERMINATION,
  NFLAG_INSURED_ON_APPROVAL
)
FROM '{db_params.DATA_FULL_PATH}previous_application.csv' DELIMITER ',' CSV HEADER;
"""

In [37]:
%%time
connector.send_sql_query(sql_data_query)

CPU times: user 7.55 ms, sys: 4.1 ms, total: 11.6 ms
Wall time: 28.2 s


In [38]:
sql_data_query = f"""
SET search_path TO credit_scoring;
COPY bureau_balance(
SK_ID_BUREAU,
MONTHS_BALANCE,
STATUS
)
FROM '{db_params.DATA_FULL_PATH}bureau_balance.csv' DELIMITER ',' CSV HEADER;
"""

In [39]:
%%time
connector.send_sql_query(sql_data_query)

CPU times: user 8.89 ms, sys: 4.23 ms, total: 13.1 ms
Wall time: 42.6 s


## Создание связей в таблицах

In [40]:
%%time
sql_schema_query = """
SET search_path TO credit_scoring;

-- Application table
ALTER TABLE application ADD PRIMARY KEY (SK_ID_CURR);

-- Bureau table
ALTER TABLE bureau ADD PRIMARY KEY (SK_ID_BUREAU);
ALTER TABLE bureau ADD CONSTRAINT fk_bureau_application FOREIGN KEY (SK_ID_CURR) REFERENCES application(SK_ID_CURR);
"""
connector.send_sql_query(sql_schema_query)

CPU times: user 5.77 ms, sys: 5.69 ms, total: 11.5 ms
Wall time: 13.2 s


In [41]:
%%time
sql_schema_query = """
SET search_path TO credit_scoring;
-- Previous Application table
ALTER TABLE previous_application ADD PRIMARY KEY (SK_ID_PREV);
ALTER TABLE previous_application ADD CONSTRAINT fk_previous_application_application FOREIGN KEY (SK_ID_CURR) REFERENCES application(SK_ID_CURR);
"""
connector.send_sql_query(sql_schema_query)

CPU times: user 5.09 ms, sys: 3.8 ms, total: 8.89 ms
Wall time: 15.1 s


In [42]:
%%time
sql_schema_query = """
SET search_path TO credit_scoring;
ALTER TABLE pos_cash_balance ADD CONSTRAINT fk_pos_cash_balance_application FOREIGN KEY (SK_ID_CURR) REFERENCES application(SK_ID_CURR);
"""
connector.send_sql_query(sql_schema_query)

CPU times: user 6.98 ms, sys: 5.23 ms, total: 12.2 ms
Wall time: 31.1 s


In [43]:
%%time
sql_schema_query = """
SET search_path TO credit_scoring;
ALTER TABLE credit_card_balance ADD CONSTRAINT fk_credit_card_balance_application FOREIGN KEY (SK_ID_CURR) REFERENCES application(SK_ID_CURR);
"""
connector.send_sql_query(sql_schema_query)

CPU times: user 7.65 ms, sys: 5.75 ms, total: 13.4 ms
Wall time: 9.32 s


In [44]:
%%time
sql_schema_query = """
SET search_path TO credit_scoring;
ALTER TABLE installments_payments ADD CONSTRAINT fk_installments_payments_application FOREIGN KEY (SK_ID_CURR) REFERENCES application(SK_ID_CURR);
"""
connector.send_sql_query(sql_schema_query)

CPU times: user 6.65 ms, sys: 5.13 ms, total: 11.8 ms
Wall time: 23.7 s


## Задания

Вывести средний доход среди всех клиентов

In [45]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT AVG(AMT_INCOME_TOTAL) AS average_income FROM application
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 12.3 ms, sys: 27.3 ms, total: 39.6 ms
Wall time: 2.22 s


,average_income
0,170116.05986


вывести минимальный и максимальный возраст среди всех клиентов

In [46]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT
MIN(ABS(DAYS_BIRTH)/365) AS min_age,
MAX(ABS(DAYS_BIRTH)/365) AS max_age
FROM credit_scoring.application;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 5.52 ms, sys: 2.9 ms, total: 8.42 ms
Wall time: 371 ms


,min_age,max_age
0,20,69


Вывести количество мужчин и женщин

In [47]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT
    CODE_GENDER,
    COUNT(*) AS total
FROM credit_scoring.application
WHERE CODE_GENDER IN ('M', 'F')
GROUP BY CODE_GENDER;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 8.25 ms, sys: 4.37 ms, total: 12.6 ms
Wall time: 518 ms


,code_gender,total
0,F,235126
1,M,121125


вывести общую сумму, количество и среднюю сумму, запрошенную клиентами в кредит с авто и без

In [48]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT
    FLAG_OWN_CAR,
    COUNT(*) AS total_loans,
    SUM(AMT_CREDIT) AS total_amount,
    AVG(AMT_CREDIT) AS average_amount
FROM credit_scoring.application
GROUP BY FLAG_OWN_CAR;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 8.43 ms, sys: 3.94 ms, total: 12.4 ms
Wall time: 433 ms


,flag_own_car,total_loans,total_amount,average_amount
0,N,235235,1.304038e+11,554317.381867
1,Y,121020,7.900278e+10,652786.569683


вывести доли клиентов с различным образованием

In [49]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT
    NAME_EDUCATION_TYPE,
    ROUND((COUNT(*)::DECIMAL / SUM(COUNT(*)) OVER ()), 6) AS proportion
FROM credit_scoring.application
GROUP BY NAME_EDUCATION_TYPE
ORDER BY proportion DESC;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 8.97 ms, sys: 5.38 ms, total: 14.4 ms
Wall time: 518 ms


,name_education_type,proportion
0,Secondary / secondary special,0.708422
1,Higher education,0.245271
2,Incomplete higher,0.033687
3,Lower secondary,0.012045
4,Academic degree,0.000575


подсчитать количество полных лет для клиентов, у которых есть во владении автомобиль и недвижимость. вывести топ 10 по возрастанию

In [50]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT
    SK_ID_CURR,
    ABS(DAYS_BIRTH) / 365 AS age_years
FROM credit_scoring.application
WHERE FLAG_OWN_CAR = 'Y' AND FLAG_OWN_REALTY = 'Y'
ORDER BY age_years ASC
LIMIT 10;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 8.38 ms, sys: 4.46 ms, total: 12.8 ms
Wall time: 518 ms


,sk_id_curr,age_years
0,283094,20
1,372716,20
2,135697,21
3,254228,21
4,107188,21
5,128287,21
6,131919,21
7,216327,21
8,301665,21
9,103397,21


Вывести тех клиентов, у кого доход на одного члена семьи в два раза больше, чем в среднем на одного члена семьи по выборке

In [51]:
%%time
sql_query = """
SET search_path TO credit_scoring;

SELECT
    SK_ID_CURR,
    AMT_INCOME_TOTAL,
    CNT_FAM_MEMBERS,
    (AMT_INCOME_TOTAL / CNT_FAM_MEMBERS) AS income_per_family_member
FROM credit_scoring.application
WHERE (AMT_INCOME_TOTAL / CNT_FAM_MEMBERS) > 2 * (
    SELECT AVG(AMT_INCOME_TOTAL / CNT_FAM_MEMBERS) FROM credit_scoring.application
)
ORDER BY SK_ID_CURR;

"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 41.2 ms, sys: 15.4 ms, total: 56.6 ms
Wall time: 1.71 s


,sk_id_curr,amt_income_total,cnt_fam_members,income_per_family_member
0,100002,202500.0,1.0,202500.0
1,100033,270000.0,1.0,270000.0
2,100053,202500.0,1.0,202500.0
3,100070,540000.0,1.0,540000.0
4,100079,225000.0,1.0,225000.0
...,...,...,...,...
27728,456184,270000.0,1.0,270000.0
27729,456189,270000.0,1.0,270000.0
27730,456209,202500.0,1.0,202500.0
27731,456228,540000.0,2.0,270000.0


вывести клиентов старше 60 лет по которым нет данных в bureau

In [52]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT a.SK_ID_CURR, floor(ABS(a.DAYS_BIRTH) / 365) AS age_years
FROM credit_scoring.application AS a
LEFT JOIN credit_scoring.bureau AS b ON a.SK_ID_CURR = b.SK_ID_CURR
WHERE b.SK_ID_CURR IS NULL
AND floor(ABS(a.DAYS_BIRTH) / 365) > 60;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 15.4 ms, sys: 5.55 ms, total: 21 ms
Wall time: 4.35 s


,sk_id_curr,age_years
0,249376,61.0
1,113210,64.0
2,234614,61.0
3,106076,64.0
4,124764,62.0
...,...,...
5051,369504,62.0
5052,352484,66.0
5053,144610,67.0
5054,432774,64.0


Вывести женщин, у которых в истории bureau было больше двух кредитов, просроченных на 61 день и более. отсортировать в порядке убывания по кол-ву таких кредитов

In [53]:
%%time
sql_query = """
SET search_path TO credit_scoring;
WITH CTE AS
  (SELECT sk_id_curr, SUM(status) AS cnt_credit
   FROM bureau
   JOIN
     (SELECT sk_id_bureau, 1 AS status
      FROM bureau_balance
      WHERE status in ('3','4','5')
      GROUP BY sk_id_bureau) AS bb ON bureau.sk_id_bureau = bb.sk_id_bureau
   GROUP BY sk_id_curr)

SELECT a.sk_id_curr,
       cnt_credit
FROM application AS a
JOIN CTE ON a.sk_id_curr = CTE.sk_id_curr
WHERE code_gender = 'F' AND cnt_credit > 2
ORDER BY cnt_credit DESC
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 11 ms, sys: 6.83 ms, total: 17.9 ms
Wall time: 36.9 s


,sk_id_curr,cnt_credit
0,279826,14
1,441227,13
2,246788,13
3,129529,10
4,164862,10
...,...,...
335,184667,3
336,182879,3
337,394391,3
338,272607,3


По данным из bureau (БКИ) расчитать долю просрочки в активных займах для каждого клиента. вывести топ 7 мужчин с наибольшей суммой просрочки, указав для них, помимо прочего, сумму активных кредитов и суммы всех кредитов (активных и закрытых).

In [54]:
%%time
sql_query = """
SET search_path TO credit_scoring;
WITH CreditInfo AS (
    SELECT
        bureau.SK_ID_CURR,
        SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS TotalOverdue,
        SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) AS ActiveCreditsSum,
        SUM(AMT_CREDIT_SUM) AS TotalCreditsSum,
        CASE 
            WHEN SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) > 0 THEN 
                SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) / 
                SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) 
            ELSE 0 
        END AS OverdueShare
    FROM bureau
    GROUP BY bureau.SK_ID_CURR
),
RankedMen AS (
    SELECT
        app.SK_ID_CURR,
        ci.TotalOverdue,
        ci.ActiveCreditsSum,
        ci.TotalCreditsSum,
        ci.OverdueShare,
        RANK() OVER (ORDER BY ci.TotalOverdue DESC) AS Rank
    FROM application AS app
    JOIN CreditInfo AS ci ON app.SK_ID_CURR = ci.SK_ID_CURR
    WHERE app.CODE_GENDER = 'M' AND ci.TotalOverdue > 0
)
SELECT SK_ID_CURR, OverdueShare, TotalOverdue, ActiveCreditsSum, TotalCreditsSum
FROM RankedMen
WHERE Rank <= 7;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 9.03 ms, sys: 5.21 ms, total: 14.2 ms
Wall time: 11.7 s


,sk_id_curr,overdueshare,totaloverdue,activecreditssum,totalcreditssum
0,435405,0.997578,3681063.00,3690000.0,4054000.5
1,427996,0.371418,1571697.00,4231615.5,7431583.5
2,394113,0.870897,1332472.50,1530000.0,1825801.0
3,266765,0.907018,1224474.90,1350000.0,1421955.0
4,167085,4.925314,780192.00,158404.5,176404.5
5,154595,0.749991,742491.00,990000.0,4245754.5
6,262411,0.538240,709669.25,1318500.0,1773000.0


## Генерация своих признаков

Молодые семьи с 2мя и больше детьми

In [55]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT
    SK_ID_CURR,
    ABS(DAYS_BIRTH) / 365 AS Age,
    CNT_CHILDREN
FROM application
WHERE
    ABS(DAYS_BIRTH) / 365 <= 27
    AND CNT_CHILDREN >= 2
ORDER BY Age ASC;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 12.7 ms, sys: 4.28 ms, total: 17 ms
Wall time: 827 ms


,sk_id_curr,age,cnt_children
0,160631,21,2
1,400044,21,2
2,263170,21,2
3,121972,21,2
4,278296,21,2
...,...,...,...
1884,399218,27,2
1885,399944,27,2
1886,400724,27,2
1887,401031,27,2


Отношение кредита к сумме платежа по кредиту

In [56]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT sk_id_curr, (amt_credit / amt_annuity) AS annuity_credit_ratio
FROM application
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 315 ms, sys: 106 ms, total: 421 ms
Wall time: 1.95 s


,sk_id_curr,annuity_credit_ratio
0,100001,27.664698
1,100005,12.824870
2,100013,9.505482
3,100028,32.130726
4,100038,19.506035
...,...,...
356250,456251,9.242325
356251,456252,22.459692
356252,456253,22.604624
356253,456254,18.317595


Средний лимит по карте по клиентам

In [57]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT
    SK_ID_CURR,
    AVG(AMT_CREDIT_LIMIT_ACTUAL) AS AverageCreditCardLimit
FROM credit_card_balance
GROUP BY SK_ID_CURR
ORDER BY AverageCreditCardLimit DESC;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 275 ms, sys: 45.4 ms, total: 320 ms
Wall time: 18.3 s


,sk_id_curr,averagecreditcardlimit
0,217324,1350000.0
1,180956,1350000.0
2,150126,1350000.0
3,135194,1350000.0
4,154897,1350000.0
...,...,...
103553,363214,0.0
103554,370279,0.0
103555,370288,0.0
103556,381912,0.0


Доля кредитного использования  — отношение общей суммы задолженности по кредитным картам к общему кредитному лимиту, доступному клиенту

In [58]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT
    SK_ID_CURR,
    CASE 
        WHEN SUM(AMT_CREDIT_LIMIT_ACTUAL) > 0 THEN 
            SUM(AMT_BALANCE) / SUM(AMT_CREDIT_LIMIT_ACTUAL)
        ELSE 
            0 
    END AS CreditUtilizationRatio
FROM credit_card_balance
GROUP BY SK_ID_CURR
ORDER BY CreditUtilizationRatio DESC;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 109 ms, sys: 18 ms, total: 127 ms
Wall time: 11 s


,sk_id_curr,creditutilizationratio
0,155492,2.138789
1,172610,2.131796
2,289267,1.564907
3,148211,1.398834
4,377094,1.286950
...,...,...
103553,179113,-0.007783
103554,385987,-0.009199
103555,203520,-0.010800
103556,172723,-0.084848


Отношение платежа по кредиту к зарплате для каждого клиента

In [59]:
%%time
sql_query = """
SET search_path TO credit_scoring;
SELECT a.sk_id_curr,
       bur.amt_annuity / a.amt_income_total AS ratio_annuity_income
FROM application a
JOIN bureau bur ON a.sk_id_curr = bur.sk_id_curr
WHERE bur.amt_annuity IS NOT NULL;
"""
connector.get_df_from_query(sql_query)

/Users/vi/DataspellProjects/shift/src/app/utils/db_connector.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: user 365 ms, sys: 109 ms, total: 474 ms
Wall time: 8 s


,sk_id_curr,ratio_annuity_income
0,125263,0.108720
1,399518,0.063978
2,399518,0.000000
3,200527,0.073292
4,200527,0.073292
...,...,...
489632,410949,0.033717
489633,115067,0.071000
489634,377669,0.000000
489635,431028,0.000000
